In [ ]:
https://docs.osmcode.org/pyosmium/latest/ref_index.html
https://osmcode.org/osmium-tool/manual.html
https://docs.osmcode.org/osmium/latest/

In [10]:
import osmium as osm
import pandas as pd
from overpass import API

api = API()


class OSMHandler(osm.SimpleHandler):
    
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []
        self.address = {'addr:housenumber': '2400', 'addr:postcode': '98144', 'addr:street': 'South Hill Street'}
        self.roads_with_latlon = []

    def node(self, n):
        dict = {}
        the_node = 0
        for tag in n.tags:
            dict[tag.k] = tag.v
        for key in self.address:
            if key not in dict:
                the_node = 1
                break
            if dict[key] != self.address[key]:
                the_node = 1
                break
        if the_node == 0:
            self.lat = n.location.lat
            self.lon = n.location.lon
            response = api.get('way["highway" ="residential"](around:95,{0},{1});'.format(n.location.lat,n.location.lon)) 
            # print('For location with coordinates {0}, {1} found way IDs:'.format(loc[0],loc[1]))
            self.roads = []
            print(response.features)
            for way in response.features:
                self.roads.append(way.id)

    def way(self, w):
       
        if w.id in self.roads:
            list = []
            # tags = []
            for road in w.nodes:
                list.append(road)
            # for tag in w.tags:
            #     tags.append(tag)
            # self.roads_with_latlon.append([list, tags])
            self.roads_with_latlon.append(list)
         # self.osm_data[w.id] =  [nodeList, tagList]
        
    # def relation(self, r):
    #     self.tag_inventory(r, "relation")

    
osmhandler = OSMHandler()
# scan the input file and fills the handler list accordingly
osmhandler.apply_file("../Downloads/map.osm", locations=True, idx='flex_mem')

# transform the list into a pandas DataFrame
# data_colnames = ['type', 'id', 'version', 'visible', 'ts', 'uid',
#                  'user', 'chgset', 'ntags', 'tagkey', 'tagvalue']
# df_osm = pd.DataFrame(osmhandler.osm_data, columns=data_colnames)
# osmhandler.roads_with_latlon


[{"geometry": {"coordinates": [], "type": "LineString"}, "id": 6487472, "properties": {"bicycle": "designated", "cycleway": "shared_lane", "highway": "residential", "name": "24th Avenue South", "tiger:cfcc": "A41", "tiger:county": "King, WA", "tiger:name_base": "24th", "tiger:name_direction_suffix": "S", "tiger:name_type": "Ave", "tiger:reviewed": "no"}, "type": "Feature"}, {"geometry": {"coordinates": [], "type": "LineString"}, "id": 481289523, "properties": {"highway": "residential", "name": "24th Avenue South", "surface": "asphalt", "tiger:cfcc": "A41", "tiger:county": "King, WA", "tiger:name_base": "24th", "tiger:name_direction_suffix": "S", "tiger:name_type": "Ave", "tiger:reviewed": "no"}, "type": "Feature"}, {"geometry": {"coordinates": [], "type": "LineString"}, "id": 481289524, "properties": {"highway": "residential", "name": "South Hill Street", "tiger:cfcc": "A41", "tiger:county": "King, WA", "tiger:name_base": "Hill", "tiger:name_direction_prefix": "S", "tiger:name_type": "

In [11]:
osmhandler.roads_with_latlon 

[[osmium.osm.NodeRef(ref=53136415, location=osmium.osm.Location(x=-1223015067, y=475869198)),
  osmium.osm.NodeRef(ref=5757431543, location=osmium.osm.Location(x=-1223015731, y=475868612)),
  osmium.osm.NodeRef(ref=2516369689, location=osmium.osm.Location(x=-1223015896, y=475868466)),
  osmium.osm.NodeRef(ref=152010665, location=osmium.osm.Location(x=-1223016159, y=475868093)),
  osmium.osm.NodeRef(ref=2516441880, location=osmium.osm.Location(x=-1223016311, y=475867646)),
  osmium.osm.NodeRef(ref=9193214107, location=osmium.osm.Location(x=-1223016316, y=475866142)),
  osmium.osm.NodeRef(ref=5440359896, location=osmium.osm.Location(x=-1223016323, y=475864945)),
  osmium.osm.NodeRef(ref=4799960441, location=osmium.osm.Location(x=-1223016328, y=475863911)),
  osmium.osm.NodeRef(ref=6632334769, location=osmium.osm.Location(x=-1223016329, y=475863758)),
  osmium.osm.NodeRef(ref=53138092, location=osmium.osm.Location(x=-1223016332, y=475863228)),
  osmium.osm.NodeRef(ref=4799960440, location

In [37]:
class RoadNode:
    def __init__(self, r):
        # self.visited = 0
        self.nodes = []
        self.intersection = 0
        if r != None:
            self.ref = r.ref
            self.lat = r.location.lat
            self.lon = r.location.lon
        else:
            self.ref = None
            self.lat = None
            self.lon = None
    def addNode(self, node):
        self.nodes = self.nodes + [node]

        


#if a road is connected (meaning one road contains a ref of another road), the road is now one big road consisting of the smaller roads

#the algo calculates routes at each point then pick the route with the shortest time to complete
road_nodes = {}
# drone_location = [osmhandler.lat, osmhandler.lon]
starting_point = None
length = 0

for roadds in osmhandler.roads_with_latlon:
    prev_node = None

    for road in roadds:
        y = abs(road.location.lat - osmhandler.lat)
        x = abs(road.location.lon - osmhandler.lon)
        distance = x + y
        print(distance)
        if starting_point == None:
            starting_point = [distance, road]
        elif starting_point[0] > distance:
            starting_point = [distance, road]
            
        #inserting to road_nodes dict.
        if prev_node == None:
            if road.ref in road_nodes:
                prev_node = road_nodes[road.ref]
            else:
                node = RoadNode(road) 
                prev_node = node
                road_nodes[road.ref] = node
            # road_node = road_nodes[road.ref]

            # prev_node.addNode(road_node)
            # road_node.addNode(prev_node)
                
            # prev_node = road_node
            # if prev_node.ref != None and prev_node.ref != road.ref:    
            
        else:
            length = length + 1
            
            if road.ref in road_nodes:
                node = road_nodes[road.ref]
            else:
                node = RoadNode(road)      
                road_nodes[road.ref] = node
            
            
            node.addNode(prev_node)
            
            prev_node.addNode(node)
            
            prev_node = node

for key in road_nodes:
    if len(road_nodes[key].nodes) > 2:
        road_nodes[key].intersection = 1



        

0.0027115999999978158
0.0027193999999965968
0.002721299999997484
0.0027102999999968347
0.0026807999999931553
0.0025308999999893445
0.0024118999999913626
0.0023089999999896804
0.002293799999996793
0.0022410999999920023
0.002192600000000766
0.0018234999999933166
0.0015089999999915449
0.001472099999993759
0.0014200000000030855
0.0013706999999882896
0.0009473999999940474
0.0005988999999928524
0.0006808999999918797
0.0006808999999918797
0.0007655999999940377
0.0014516999999898417
0.0015549999999961983
0.0006808999999918797
0.0005485999999876867
0.0002193999999988705
0.0007591999999974064
0.0008786000000000627
0.0016802999999825374
0.0015112999999971066
0.001316799999990792
0.0010335999999924184
0.0007746999999937998
0.0006808999999918797
0.002430700000005004
0.001688000000015677
0.0016335000000182731
0.0015773000000152138
0.0008027999999953295
0.0008786000000000627
0.0009349999999983538
0.0015290999999990618
0.0016518000000118604
0.0017341000000001827
0.0024362000000053285
0.003161099999999

In [38]:
print(starting_point)

[0.0002193999999988705, osmium.osm.NodeRef(ref=3663412746, location=osmium.osm.Location(x=-1223009966, y=475844989))]


In [39]:
# def fly_drone_to_dest(start, dest, visited):
#     start = 0
#     end = 0
#     for i, r in enumerate(visited):
#         if r == start:
#             start = i
#         elif r == dest:
#             end = i
#     path = []
#     if start > end:
#         for i in range(start, end, -1):
#             path.append(visited[i])
#     else:
#         path = visited[start:end]
#     return path

def instructions_dfs(roadNode, visited, intersection_point):
    
    visited = visited + [roadNode]

    if roadNode.intersection == 1:
        intersection = roadNode
    else:
        intersection = intersection_point
    
    for r in roadNode.nodes:
        if r in visited:
            continue
        
        visited = instructions_dfs(r, visited, intersection)
        
        if intersection_point != None or roadNode.intersection == 1:
            visited = visited + [roadNode]

    return visited
     
all_dir = []

# for key in road_nodes: 
#     
#     if road_nodes[key] in visited: 
#         continue
#     # visited = visited + [road_nodes[key]]
#     list = instructions_dfs(road_nodes[key], visited, road_nodes[key])
#     all_dir.append(list)

visited = []
for node in road_nodes[starting_point[1].ref].nodes:
    list = instructions_dfs(road_nodes[starting_point[1].ref], visited, road_nodes[starting_point[1].ref])
    if len(list) > length:
        all_dir = all_dir + [list]
    

In [40]:
smallest_route = None
for route in all_dir:
    if smallest_route == None:
        smallest_route = route
    elif len(smallest_route) > len(route):
        smallest_route = route

storeable_smallest_route = []
for route in smallest_route:
    storeable_smallest_route.append([route.lon, route.lat])


In [ ]:
https://stackoverflow.com/questions/35935670/share-variables-between-different-jupyter-notebooks

In [41]:
%store storeable_smallest_route


Stored 'storeable_smallest_route' (list)


In [32]:
# starting_point = [starting_point[1].location.lat, starting_point[1].location.lon]
# %store starting_point

In [42]:
%store

Stored variables and their in-db values:
starting_point                       -> [47.5844989, -122.3009966]
storeable_smallest_route             -> [[-122.3009966, 47.5844989], [-122.3015205, 47.584
